# Question 1

In [ ]:
import sys
from abc import ABC, abstractmethod
from collections import deque
import numpy as np

################################################################################

class Node:
  PRECISION = 2
  PADDING = 0

  class Kind:
    LEAF = 0b00
    RGHT = 0b01
    LEFT = 0b10
    BOTH = 0b11

  def __init__(self, value, aboveNode=None, leftNode=None, rightNode=None):
    self.value = value
    self.aboveNode:Node = aboveNode
    self.leftNode:Node = leftNode
    self.rightNode:Node = rightNode
    self.height = 1

  def __repr__(self):
    # Node.PADDING = 6; return f"{{:1.{Node.PRECISION}e}}".format(self.value)
    # Node.PADDING = 2; return f"{{:0{Node.PRECISION}d}}@{self.height}".format(self.value)
    Node.PADDING = 0; return f"{{:0{Node.PRECISION}d}}".format(self.value)

  def __eq__(self, node):
    return self.value == node.value

  def __lt__(self, node):
    return self.value < node.value
  
  def __gt__(self, node):
    return self.value > node.value
      
  def is_trunk(self):
    return self.aboveNode is None
  
  def determine_kind(self):
    return ((self.leftNode is not None) << 1) | (self.rightNode is not None)
  
  def determine_balance(self):
    match self.determine_kind():
      case Node.Kind.LEAF:
        return 0
      case Node.Kind.RGHT:
        return -self.rightNode.height
      case Node.Kind.LEFT:
        return self.leftNode.height
      case Node.Kind.BOTH:
        return self.leftNode.height - self.rightNode.height
      case _:
        raise ValueError(f"Indeterminate node kind ({self.determine_kind()})")

  def update_height(self):
    lh = 0 if not self.leftNode else self.leftNode.height
    rh = 0 if not self.rightNode else self.rightNode.height
    self.height = 1 + max(lh,rh)
    if self.aboveNode:
      self.aboveNode.update_height()

  def update_child(self, node):
    if node < self:
      self.leftNode = node
    elif node > self:
      self.rightNode = node
    else:
      raise ValueError(f"""Equal parent and child nodes
                       \n\tparent: {self}
                       \n\tchild: {node}""")

################################################################################

class Tree(ABC):
  
  class Chars:
    DESC = '-'  # '\u2500'
    BOTH = '|'  # '\u2524'
    LEFT = 'V'  # '\u2510'
    RHGT = 'A'  # '\u2518'

  @abstractmethod
  def __init__(self, node:Node=None):
    self.root = node

  @abstractmethod
  def find(self, val):
    raise NotImplementedError
  
  @abstractmethod
  def insert(self, val):
    raise NotImplementedError

  @abstractmethod
  def delete(self, val):
    raise NotImplementedError

  @staticmethod
  def illustrate(node: Node, depth=0):
    lead = " " * (Node.PRECISION + Node.PADDING + 3) * depth
    if node is not None:
      match node.determine_kind():
        case Node.Kind.LEAF:
          print(lead + str(node))
        case Node.Kind.RGHT:
          Tree.illustrate(node.rightNode, depth+1)
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.RHGT)
        case Node.Kind.LEFT:
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.LEFT)
          Tree.illustrate(node.leftNode, depth+1)
        case Node.Kind.BOTH:
          Tree.illustrate(node.rightNode, depth+1)
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.BOTH)
          Tree.illustrate(node.leftNode, depth+1)
        case _:
          raise ValueError(f"Unknown node kind ({node.determine_kind()})")
        
  @staticmethod
  def inorder_traversal(node:Node, v:list):
    if not node:
      return
    if node.leftNode:
      Tree.inorder_traversal(node.leftNode, v)
    v.append(node.value)
    if node.rightNode:
      Tree.inorder_traversal(node.rightNode, v)
      
################################################################################
class BinaryTree(Tree):
  def __init__(self, node:Node=None):
    """
    Initialize tree
    
    hint: an empty tree has a None root
    """
    super().__init__(node)

  def find(self, val:int):
    """
    Locate node equal to val
    
    hint: this function is not test directly,
        but may be useful in other methods 
        if it returns some kind of node whether 
        found or not
    """
    current = self.root
    while current:
        if val < current.value:
            current = self.root.leftNode
        elif val > current.value:
            current = self.root.rightNode
        else:
            return current
    return None
    
  def insert(self, val:int):
    """
    Insert val as a node into tree
    
    hint: do not reinsert an existing node in tree
    """
    new_node = Node(val)
    if self.root is None:
        self.root = new_node
        return
    
    current = self.root
    while True:
        if val < current.value:
            if current.leftNode is None:
                current.leftNode = new_node
                return
            else:
                current = current.leftNode
        elif val > current.value:
            if current.rightNode is None:
                current.rightNode = new_node
                return
            else:
                current = current.rightNode
        else:
            return
    
    
    
  def delete(self, val:int):
    """
    Delete node from tree equal to val, if it exists
    """
    current = self.root
    parent = None
    is_left_child = False

    # find the node to delete
    while current and current.value != val:
        parent = current
        if val < current.value:
            current = current.leftNode
            is_left_child = True
        else:
            current = current.rightNode
            is_left_child = False

    if current is None:  # Value not found in tree
        return

    # when node is leaf node
    if current.leftNode is None and current.rightNode is None:
        if current == self.root:
            self.root = None
        elif is_left_child:
            parent.leftNode = None
        else:
            parent.rightNode = None

    # when node has only right child
    elif current.leftNode is None:  
        if current == self.root:
            self.root = current.rightNode
        elif is_left_child:
            parent.leftNode = current.rightNode
        else:
            parent.rightNode = current.rightNode

    # when node has only left child
    elif current.rightNode is None: 
        if current == self.root:
            self.root = current.leftNode
        elif is_left_child:
            parent.leftNode = current.leftNode
        else:
            parent.rightNode = current.leftNode

    # when node has both left and right child
    else:
        successor = current.rightNode
        successor_parent = current
        while successor.leftNode:
            successor_parent = successor
            successor = successor.leftNode

        # find the left most node in the right binary tree and replace current to the leftmost node
        current.value = successor.value

        # Delete the successor
        if successor.rightNode:
            if successor_parent == current:
                successor_parent.rightNode = successor.rightNode
            else:
                successor_parent.leftNode = successor.rightNode
        else:
            if successor_parent == current:
                successor_parent.rightNode = None
            else:
                successor_parent.leftNode = None

################################################################################

def test(tree:Tree, n, p=0.0, seed=0, verbose=False):
  valueRange = (0,10**Node.PRECISION)
  seed = np.random.randint(0,1e6) if not seed else seed
  np.random.seed(seed)
  t = tree()
  values = np.random.randint(*valueRange,n)
  evicts = np.random.choice(values, round(p*len(values)))
  for v in values:
    t.insert(v)
  for e in evicts:
    t.delete(e)
  
  vals = []
  Tree.inorder_traversal(t.root, vals)
  code = sorted(set(values).difference(set(evicts)))
  
  print("="*100)
  if verbose:
    print("{} | n={} | uniq(n)={} | m={} | uniq(m)={} | range = [{},{}) | seed = {}".format(
      tree.__name__, n, len(set(values)), len(evicts), len(set(evicts)), *valueRange, seed))
    print("-"*100)
  t.illustrate(t.root)
  print("="*100)

  assert vals == code, "Inconsistent tree nodes"


def main():
    lines = sys.stdin.readlines()
    assert len(lines)==5, "bad input"
    t = lines.pop(0).strip()
    n = int(lines.pop(0).strip())
    p = float(lines.pop(0).strip())
    s = int(lines.pop(0).strip())
    P = int(lines.pop(0).strip())
    Node.PRECISION = P
    test(globals()[t], n, p, s, True)
        
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print(f"ERROR: {e}")

# Question 2

In [ ]:
import sys
from abc import ABC, abstractmethod
from collections import deque
import numpy as np

################################################################################

class Node:
  PRECISION = 2
  PADDING = 0

  class Kind:
    LEAF = 0b00
    RGHT = 0b01
    LEFT = 0b10
    BOTH = 0b11

  def __init__(self, value, aboveNode=None, leftNode=None, rightNode=None):
    self.value = value
    self.aboveNode:Node = aboveNode
    self.leftNode:Node = leftNode
    self.rightNode:Node = rightNode
    self.height = 1

  def __repr__(self):
    # Node.PADDING = 6; return f"{{:1.{Node.PRECISION}e}}".format(self.value)
    # Node.PADDING = 2; return f"{{:0{Node.PRECISION}d}}@{self.height}".format(self.value)
    Node.PADDING = 0; return f"{{:0{Node.PRECISION}d}}".format(self.value)

  def __eq__(self, node):
    return self.value == node.value

  def __lt__(self, node):
    return self.value < node.value
  
  def __gt__(self, node):
    return self.value > node.value
      
  def is_trunk(self):
    return self.aboveNode is None
  
  def determine_kind(self):
    return ((self.leftNode is not None) << 1) | (self.rightNode is not None)
  
  def determine_balance(self):
    match self.determine_kind():
      case Node.Kind.LEAF:
        return 0
      case Node.Kind.RGHT:
        return -self.rightNode.height
      case Node.Kind.LEFT:
        return self.leftNode.height
      case Node.Kind.BOTH:
        return self.leftNode.height - self.rightNode.height
      case _:
        raise ValueError(f"Indeterminate node kind ({self.determine_kind()})")

  def update_height(self):
    lh = 0 if not self.leftNode else self.leftNode.height
    rh = 0 if not self.rightNode else self.rightNode.height
    self.height = 1 + max(lh,rh)
    if self.aboveNode:
      self.aboveNode.update_height()

  def update_child(self, node):
    if node < self:
      self.leftNode = node
    elif node > self:
      self.rightNode = node
    else:
      raise ValueError(f"""Equal parent and child nodes
                       \n\tparent: {self}
                       \n\tchild: {node}""")

################################################################################

class Tree(ABC):
  
  class Chars:
    DESC = '-'  # '\u2500'
    BOTH = '|'  # '\u2524'
    LEFT = 'V'  # '\u2510'
    RHGT = 'A'  # '\u2518'

  @abstractmethod
  def __init__(self, node:Node=None):
    self.root = node

  @abstractmethod
  def find(self, val):
    raise NotImplementedError
  
  @abstractmethod
  def insert(self, val):
    raise NotImplementedError

  @abstractmethod
  def delete(self, val):
    raise NotImplementedError

  @staticmethod
  def illustrate(node: Node, depth=0):
    lead = " " * (Node.PRECISION + Node.PADDING + 3) * depth
    if node is not None:
      match node.determine_kind():
        case Node.Kind.LEAF:
          print(lead + str(node))
        case Node.Kind.RGHT:
          Tree.illustrate(node.rightNode, depth+1)
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.RHGT)
        case Node.Kind.LEFT:
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.LEFT)
          Tree.illustrate(node.leftNode, depth+1)
        case Node.Kind.BOTH:
          Tree.illustrate(node.rightNode, depth+1)
          print(lead + str(node) + Tree.Chars.DESC*3 + Tree.Chars.BOTH)
          Tree.illustrate(node.leftNode, depth+1)
        case _:
          raise ValueError(f"Unknown node kind ({node.determine_kind()})")
        
  @staticmethod
  def inorder_traversal(node:Node, v:list):
    if not node:
      return
    if node.leftNode:
      Tree.inorder_traversal(node.leftNode, v)
    v.append(node.value)
    if node.rightNode:
      Tree.inorder_traversal(node.rightNode, v)
      
################################################################################
class AVLTree(Tree):
    def __init__(self, node: Node = None):
        super().__init__(node)


    def find(self, val):
        def _find_recursive(node, val):
            if node is None or node.value == val:
                return node
            elif val < node.value:
                return _find_recursive(node.leftNode, val)
            else:
                return _find_recursive(node.rightNode, val)

        return _find_recursive(self.root, val)

    def insert(self, val):
        # if already exist: no need to insert
        if self.find(val) is not None:
            return
        else:
            self.root = self._insert_node(self.root, val)

    def _insert_node(self, root, val):
        # binary tree
        if not root:
            return Node(val)
        elif val < root.value:
            root.leftNode = self._insert_node(root.leftNode, val)
        else:
            root.rightNode = self._insert_node(root.rightNode, val)

        # Update height and balance
        root.update_height()
        balance = self._get_balance(root)

        # If unbalanced
        # Left Left Case
        if balance > 1 and val < root.leftNode.value:
            return self._rotate_right(root)
        # Right Right Case
        if balance < -1 and val > root.rightNode.value:
            return self._rotate_left(root)
        # Left Right Case
        if balance > 1 and val > root.leftNode.value:
            root.leftNode = self._rotate_left(root.leftNode)
            return self._rotate_right(root)
        # Right Left Case
        if balance < -1 and val < root.rightNode.value:
            root.rightNode = self._rotate_right(root.rightNode)
            return self._rotate_left(root)

        return root

    def delete(self, val):
        # node does not exists: no need to delete
        if self.find(val) is None:
            return
        else:
            self.root = self._delete_node(self.root, val)

    def _delete_node(self, root, val):
        # binary tree
        if not root:
            return root
        if val < root.value:
            root.leftNode = self._delete_node(root.leftNode, val)
        elif val > root.value:
            root.rightNode = self._delete_node(root.rightNode, val)
        else:
            if root.leftNode is None:
                return root.rightNode
            elif root.rightNode is None:
                return root.leftNode

            temp = self._min_value_node(root.rightNode)
            root.value = temp.value
            root.rightNode = self._delete_node(root.rightNode, temp.value)

        # Update height and balance
        root.update_height()
        balance = self._get_balance(root)

        # If unbalanced
        # Left Left Case
        if balance > 1 and self._get_balance(root.leftNode) >= 0:
            return self._rotate_right(root)
        # Left Right Case
        if balance > 1 and self._get_balance(root.leftNode) < 0:
            root.leftNode = self._rotate_left(root.leftNode)
            return self._rotate_right(root)
        # Right Right Case
        if balance < -1 and self._get_balance(root.rightNode) <= 0:
            return self._rotate_left(root)
        # Right Left Case
        if balance < -1 and self._get_balance(root.rightNode) > 0:
            root.rightNode = self._rotate_right(root.rightNode)
            return self._rotate_left(root)

        return root

    # Utility methods for rotations and balance calculations
    def _rotate_left(self, y):
        x = y.rightNode
        T2 = x.leftNode
        x.leftNode = y
        y.rightNode = T2
        y.update_height()
        x.update_height()
        return x

    def _rotate_right(self, x):
        y = x.leftNode
        T2 = y.rightNode
        y.rightNode = x
        x.leftNode = T2
        x.update_height()
        y.update_height()
        return y

    def _get_balance(self, node):
        if not node:
            return 0
        return node.determine_balance()

    def _min_value_node(self, node):
        current = node
        while current.leftNode is not None:
            current = current.leftNode
        return current

# Additional code for the Node class and the abstract Tree class



  # Other Hints:
  # - AVL trees require special methods for left and right rotations
  # - AVL trees are sensitive to the relative heights of any Node's subtrees
  # - AVL trees should iteraively or recursivly "fixup" node heights and imbalances
  #     after each insertion and deletion
    
################################################################################

def test(tree:Tree, n, p=0.0, seed=0, verbose=False):
  valueRange = (0,10**Node.PRECISION)
  seed = np.random.randint(0,1e6) if not seed else seed
  np.random.seed(seed)
  t = tree()
  values = np.random.randint(*valueRange,n)
  evicts = np.random.choice(values, round(p*len(values)))
  for v in values:
    t.insert(v)
  for e in evicts:
    t.delete(e)
  
  vals = []
  Tree.inorder_traversal(t.root, vals)
  code = sorted(set(values).difference(set(evicts)))
  
  print("="*100)
  if verbose:
    print("{} | n={} | uniq(n)={} | m={} | uniq(m)={} | range = [{},{}) | seed = {}".format(
      tree.__name__, n, len(set(values)), len(evicts), len(set(evicts)), *valueRange, seed))
    print("-"*100)
  t.illustrate(t.root)
  print("="*100)

  assert vals == code, "Inconsistent tree nodes"


def main():
    lines = sys.stdin.readlines()
    assert len(lines)==5, "bad input"
    t = lines.pop(0).strip()
    n = int(lines.pop(0).strip())
    p = float(lines.pop(0).strip())
    s = int(lines.pop(0).strip())
    P = int(lines.pop(0).strip())
    Node.PRECISION = P
    test(globals()[t], n, p, s, True)
        
if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print(f"ERROR: {e}")